In [ ]:
import tweepy 
!pip install wget
import wget
import datetime;
import os;

In [ ]:
# directory = "./pictures"
# test = os.listdir(directory)

# print(test)
# for item in test:
#     if item.endswith(".timestamp"):
#         os.remove(os.path.join( directory, item ))

In [9]:
ckey = "Qz32iCclmyA6cKuBHHzw0Wxb2" 
csecret = "rGptlYy0bfOesVphsVWDz2rnfPsQcyL6Xt6zbJIf59ZMxdSt98" 
atoken = "1703690599-mOGCfpnhpysEqgeqeRoKEg2Jl4UIj2OUUXHc5Ug" 
asecret = "xZfCcEsEea9qJ1mFHB52Si1PBtl23RIYnkAJOqvYEZFlw" 

OAUTH_KEYS = {'consumer_key':ckey, 'consumer_secret':csecret, 
 'access_token_key':atoken, 'access_token_secret':asecret} 
auth = tweepy.OAuthHandler(OAUTH_KEYS['consumer_key'], OAUTH_KEYS['consumer_secret']) 
api = tweepy.API(auth, wait_on_rate_limit=True) 

def create_folder(output_folder):
    "Create a folder if it doesn't exist. Return modification time of .timestamp if it does."
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        timestamp = 0
    else:
        try:
            timestamp = os.stat(os.path.join(output_folder, '.timestamp')).st_mtime
        except:
            timestamp = 0
    return timestamp

def get_media(media):
    a=[ { 'filename': f"{item['id_str']}.jpg", 
          'url': f"{item['media_url']}?format=jpg&name=large" }
        for item in media if item['type'] == 'photo' ]

    for item in media:
        if item['type'] == 'photo': continue
        from pprint import pprint as pp
        pp("Unhandled media type")
        pp(item["type"])

    return a

# It returns [] if the tweet doesn't have any media
def tweet_media_urls(tweet_status):
    # At least one image
    if 'media' in tweet_status.entities:
        # Grabbing all pictures
        media = tweet_status.extended_entities['media']

        return get_media(media)
    else:
        return {}

def download_images(status, num_tweets, output_folder):
    timestamp = create_folder(output_folder)
    ts_file = os.path.join(output_folder, '.timestamp')
    downloaded = 0

    for tweet_status in status:
        if downloaded >= num_tweets:
            print('Stopping after downloading ' + str(num_tweets) + ' tweets.' )
            break

        # Metadata for creating the filename, image comment, timestamp
        created = tweet_status.created_at.strftime('%y-%m-%d at %H.%M.%S %Z')
        tweet_id = tweet_status.id_str
        full_text = tweet_status.full_text

        # Creation time of tweet as seconds.nanoseconds since The Epoch.
        ctime = tweet_status.created_at.strftime('%s') 
        ctime = float(ctime)

        if ctime < timestamp:
            print('Stopping at ' + created + ' which is older than .timestamp' )
            return

        for media_info in tweet_media_urls(tweet_status):
            # Download each media URL, if the file doesn't exist already.
            file_name = media_info['filename']
            media_url = media_info['url']

            output_file = os.path.join(output_folder, file_name)
            if os.path.exists(output_file):
                print(f"Skipping existing file: {file_name}")
            else:
                print(media_url)
                print(full_text)
                print(output_file)

                wget.download(media_url, out=output_file)
                downloaded += 1

                # Set modification time to the tweet creation time.
                os.utime(output_file, (ctime, ctime))

                # Touch the .timestamp file 
                os.close(os.open(ts_file, os.O_CREAT))

    print("End of tweet statuses")
    print(downloaded + " images were downloaded.")


def download_images_by_tag(api, tag, retweets, replies, num_tweets, output_folder):
    status = tweepy.Cursor(api.search, '#' + tag, include_rts=retweets, exclude_replies=replies,
                           tweet_mode='extended').items()
    download_images(status, num_tweets, output_folder)

hashtags = ['ukrainerussianwar', 'refugees', 'ukrainewar', 'ukraine', 'russia', 'putin']
for hashtag in hashtags:
  download_images_by_tag(api, hashtag, False, True, 10000, './pictures')


Streaming output truncated to the last 5000 lines.
http://pbs.twimg.com/media/FPiShWIVcAYYbAX.jpg?format=jpg&name=large
After this, it’s time for the gloves to come off.

#Ukraine #RussianWarCrimes #russianinvasion https://t.co/qRlcrhQdwS
./pictures/1511118776556679174.jpg
Skipping existing file: 1511118744898183169.jpg
http://pbs.twimg.com/media/FPh-NcIXIAgNLNw.jpg?format=jpg&name=large
RT @Rawle95Rawle951: #Ukraine https://t.co/YmNXPlSINk
./pictures/1511096444337463304.jpg
'Unhandled media type'
'video'
'Unhandled media type'
'animated_gif'
'Unhandled media type'
'video'
Skipping existing file: 1511116832144596998.jpg
'Unhandled media type'
'video'
'Unhandled media type'
'video'
'Unhandled media type'
'video'
Skipping existing file: 1511118685288771592.jpg
Skipping existing file: 1511117793139253249.jpg
Skipping existing file: 1510664341858791434.jpg
'Unhandled media type'
'video'
'Unhandled media type'
'video'
http://pbs.twimg.com/media/FPiSbA2WQAQi5UC.jpg?format=jpg&name=large
@har

In [ ]:
!zip -r /content/twitterpictures.zip /content/pictures
from google.colab import files
